# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

Classification is for discrete inputs and outputs, whereas regression is for continuous inputs and outputs.  For the student intervention dataset, the final classification is binary, either 'pass' or 'fail', with a fail prediction indicating early intervention is necessary.  The feature data are also discrete, being either binary (e.g. sex), discrete ordinal (e.g. age), or categorical (e.g. Mjob), so classification is more appropriate than regression as the supervised learning method for this problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
PASSED = student_data[student_data['passed'] == 'yes']
n_passed = float(len(PASSED))

# TODO: Calculate failing students
FAILED = student_data[student_data['passed'] == 'no']
n_failed = float(len(FAILED))

# TODO: Calculate graduation rate
grad_rate = 100.0 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265.0
Number of students who failed: 130.0
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [10]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state = 222)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

The classifier versions of SVM, Bagging, and LogisticRegression are three appropriate models for this problem.  For this dataset, hey are the best performing versions with default settings from each of the following families of models, respectively: Instance-Based Learning (SVM), Ensemble Learning (Bagging), and Linear Models (LogisticRegression). These three are typically among the first to be tried due to their ease of use, simplicity of tuning (particularly Bagging), transparency (LogisticRegression) and likelihood of good performance (all three, see details below). With our small dataset, it was actually practical to actually try every single one of the classifiers in the above supervised learning list to see how well they did on this dataset out of the box.  These three classifiers are among the best performing.  Naive Bayes and Decision Trees were also tested but they significantly underperform the above models using default settings with test F1 scores in the mid 70%.  

*Note: Stochastic Gradient Descent, also a linear model, outperforms LogisticRegression with default settings on this dataset with 100 and 200 data points but fails to predict any outcome for 300 data points; so LogisticRegression is chosen to represent linear models instead. Also, while the other three ensemble options are all known to work better in general than Bagging, with the default settings on this dataset Bagging gives the best F1 score by over 2% so it is chosen... the data has spoken.*

**SVM**

SVM has been widely used in countless manners since its invention in the late 1990s.  One SVM application list can be found here: http://www.clopinet.com/SVM.applications.html.  From the lectures, SVM works well with complicated, high dimensional datasets where the data is cleanly separated, being a maximum margin type of classifier.  However, because the training time is cubic to dataset size, training of large datasets can be too costly in time and/or processing resources, and if the data is noisy, then SVMs could overfit on the noise. 

Another list of SVM strengths and weaknesses can be found here: http://www.cs.rutgers.edu/~mlittman/courses/ml04/svm.pdf.  Per this document,

Strengths of SVM includes:

– Training is relatively easy , with no local optimal, unlike in neural networks

– It scales relatively well to high dimensional data

– Tradeoff between classifier complexity and error can be controlled explicitly

– Non-traditional data like strings and trees can be used as input to SVM, instead of feature vectors

Weaknesses of SVM include:

– Need a “good” kernel function, meaning that finding a good kernel function for a particular dataset requires expertise.

SVM, being a maximum margin classifier, is essentially a binary classifier which directly applicable to the problem here, though it could also work for multi-class classification.  For small samples and non-text classification SVM should be one of the first classifiers to try due to its simplicity of use and likelihood of good performance out of of the box.

**Bagging**

The paper at http://www.sciencedirect.com/science/article/pii/S1877050915007103 describes an application of Bagging Classifier for credit card frautd detection.  From the brief lecture on bagging, the main advantage discussed is the separate fitting of different data subsets and averaging of these sub-fits that makes the overall fit less affected by outliers and less likely to overfit to any one part of the data.

I found a list of Bagging strengths and weaknesses at http://www.datasciencecentral.com/m/blogpost?id=6448529%3ABlogPost%3A428995

Strengths:

    Robust against outliers and noise. 
    
    Reduces variance and typically avoids overfitting.
    
    Easy to use with limited well established default parameters.  Works well off the shelf requiring little additional tuning. 
    
    Fast run time.
    
    Since it is tree based it confers most of the advantages of trees like handling missing values, automatic selection of variable importance, and accomodating highly non-linear interactions.

Weakness:

    Can be slow to score as complexity increases.
    
    Lack of transparancy due to the complexity of multiple tree

BaggingClassifier is another easy to use classifier that could work well regardless of outliers and noise.  It's a good approach to try first because it has the potential to yield good result with relatively little tuning due to only a few parameters that have known good default values.  It's tree-based approach can handle non-linear relationships if they are present.  With our 30 features and potential datapoints in the thousands its slowness from increased complexity won't be much of an issue.

**LogisticRegression**

The paper at http://www.math.montana.edu/grad/writing-projects/09sulock.pdf describes an application of logistic regression to predicting college enrollment. LogisticRegression wasn't covered by the source materials, but in separate readings I gather that this is a classifier that determines the probability of value of a binary dependent variable based on a number of input independent variables using the logistic distribution function.  This is the exactly the form of the student intervention problem where there is a number of (presumably, but not proven) independent variables (sex, age, Mjob, etc.) that determines the likelihood of a binary dependent variable (passed, yes or no).

A list of strengths and weaknesses of logistric regression can be found at http://www.edvancer.in/logistic-regression-vs-decision-trees-vs-svm-part2/


Logistic Regression Pros:

    Convenient probability scores for observations

    Efficient implementations available across tools
    
    Multi-collinearity is not really an issue and can be countered with L2 regularization to an extent
    
    Wide spread industry comfort for logistic regression solutions [ oh that’s important too!]

Logistic Regression Cons:

    Doesn’t perform well when feature space is too large
    
    Doesn’t handle large number of categorical features/variables well
    
    Relies on transformations for non-linear features
    
    Relies on entire data [ Not a very serious drawback I’d say]
    
I've also read that logistic regression will overfit if there are dependencies among the features and will not produce good models if one or more relevant features are omitted from the data.  Hopefully the features are sufficiently relevant to the dependent variables 'passed' so predictions with this model will be useful.

LogisticRegression is often recommended as one of the first classifier to try since it can work quickly and transparently.  It's widely used for this reason.  With 30 features there could be a performance issue with LogisticRegression but empirically it still seems to work well on this dataset and much better than decision trees.  It's top level performance among all the various classifiers on this dataset indicates that its weaknesses are not substantially in play here.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [11]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [14]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegressionCV

# TODO: Initialize the three models
clf_A = SVC(random_state=1001)
clf_B = BaggingClassifier(random_state=1001)
clf_C = LogisticRegressionCV(random_state=1001)

# TODO: Set up the training set sizes
# TODO: Execute the 'train_predict' function for each classifier and each training set size
# TODO: Execute the 'train_predict' function for each classifier and each training set size
# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8701.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8375.
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8822.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8366.
Training a SVC using a training set size of 300. . .
Trained model in 0.0090 seconds
Made predictions in 0.0060 seconds.
F1 score for training set: 0.8739.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8205.

BaggingClassifier: 

Training a BaggingClassifier using a training set size of 100. . .
Trained model in 0.0240 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.9851.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7910.
Training a BaggingClassifier using a training set size of 200. .

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020 s         |       0.0010 s         |      0.8701      |     0.8375      |
| 200               |        0.0040 s         |       0.0020 s         |      0.8822      |     0.8366      |
| 300               |        0.0090 s         |       0.0020 s         |      0.8739      |     0.8205      |

** Classifer 2 - BaggingClassifier   

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0230 s         |       0.0020 s         |      0.9851      |     0.7910      |
| 200               |        0.0280 s         |       0.0010 s         |      0.9846      |     0.7424      |
| 300               |        0.0330 s         |       0.0010 s         |      0.9975      |     0.8321      |

** Classifer 3 - LogisticRegression 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.4550 s         |       0.0000 s         |      0.9000      |     0.8194      |
| 200               |        0.4420 s         |       0.0000 s         |      0.8311      |     0.8138      |
| 300               |        0.3990 s         |       0.0000 s         |      0.8117      |     0.8267      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

SVM looks promissing in terms of out of the bag performance, but the 400-count *'academic'* dataset that we're working with is likely much smaller if we're in any decent-sized school district where the appropriately-aged student count could be in the thousands and tens of thousands.  With that kind of dataset size and SVM training time is cubic with data point count, training time/cost could get prohibitive.  Given that both bagging and logistic regression both surpasses SVM performance on all 300 training data points, I will choose to use bagging with logistic regression as base estimator instead of the default decision tree (in sklearn). This approach should leverage logistic regression's natural fit to this binary classification problem while also reduce its tendency to overfit on irrelevant data while keeping training time and computational costs reasonable.  Also, the logistic regression will use the 'liblinear' solver to reduce training time.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

In this student intervention study, we are interested in building a model of how various student attributes such as age, sex, family size, economic status, parent employment, etc. affects the probability that a student would pass or fail. Building such a model allow us to input values of these attributes from future students into the model to predict whether the student is at risk of failing and thus should be the target of early intervention to empower the student to pass.

Our dataset and output are categorical, so we need classifier models instead of regression models, which work on continuous numerical data.  However, determining analytically which particular classifier is most suitable to our dataset (or any particular dataset) is hard. So, we choose to approach the problem empirically by trying several classification methods on different sizes of our dataset. From how well these models perform out of the box, we narrow down the choice to three classifiers that performs best on our dataset.  From these three classifiers we will select the best one to tune for further performance improvement.

It turns out that the classifier chosen is a combination of two of the three out-of-the-box approaches, bagging and logistic regression.  Bagging is an approach that applies a sub-classifier on random subsets of the data and then combine them in a mathematically reasonable way to produce the final classification.  By working separately on subsets of the data, this approach is less susceptible to distortion by outliers and can model local trends in data behaviors.  Out of the box, the sub-classifier used by bagging is the decision tree classifier.  Empirically, decision trees did not perform as well as the second model, logistic regression, so we will use logistic regression instead as the sub-classifier with bagging.  In other words, logistic regression will be the classifier that will be applied to the random subsets of the data that will later be *bagged* together to determine the final result. Applying logistic regression to these random data subsets can be done independently and thus can be parallelized across multiple processors that are common in current computers to reduce training time.

Logistic regression is a way of determining how various independent factors, such as the various attributes of our students, could affect a dependent variable, in our case whether a student passes or fails.  This model is designed for binary outcomes such as the passes yes or no in our study. While this approach is susceptible to outliers and overfitting to irrelevant data if they are present, bagging this classifier on random subsets should reduce the impact of these weaknesses.  Thus, combining these two complementary approaches and then automatically searching for the optimal settings for these models should improvement on their out of the box performance and produce a combined model with accurate predictive capability.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [15]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = { 'n_estimators' : [ 5, 10, 15],
               'max_samples' : [0.1, 0.2, 0.3],
               'max_features' : [0.6, 0.8, 1.0] }

# TODO: Initialize the classifier
base_clf = LogisticRegressionCV(random_state=1001, solver='liblinear')
main_clf = BaggingClassifier(base_estimator=base_clf, random_state=1001)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=main_clf, param_grid=parameters, scoring=f1_scorer, verbose=3)

# TODO: Fit the grid search object to the training data and find the optimal parameters
t0 = time()
grid_obj = grid_obj.fit(X_train, y_train)
print ''
print 'Grid search took {:.2f} seconds.'.format(time() - t0)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print ''
print clf

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] max_features=0.6, max_samples=0.1, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=5, score=0.712121 -   0.0s
[CV] max_features=0.6, max_samples=0.1, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=5, score=0.784810 -   0.0s
[CV] max_features=0.6, max_samples=0.1, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=5, score=0.696970 -   0.0s
[CV] max_features=0.6, max_samples=0.1, n_estimators=10 ..............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=10, score=0.779221 -   0.1s
[CV] max_features=0.6, max_samples=0.1, n_estimators=10 ..............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=10, score=0.762500 -   0.1s
[CV] max_features=0.6, max_samples=0.1, n_estimators=10 ..............


C:\Program Files\Anaconda2\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


[CV]  max_features=0.6, max_samples=0.1, n_estimators=10, score=0.760563 -   0.1s
[CV] max_features=0.6, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=15, score=0.787500 -   0.1s
[CV] max_features=0.6, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=15, score=0.797468 -   0.1s
[CV] max_features=0.6, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.6, max_samples=0.1, n_estimators=15, score=0.712121 -   0.1s
[CV] max_features=0.6, max_samples=0.2, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.2, n_estimators=5, score=0.802721 -   0.0s
[CV] max_features=0.6, max_samples=0.2, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.2, n_estimators=5, score=0.795031 -   0.0s
[CV] max_features=0.6, max_samples=0.2, n_estimators=5 ...............
[CV]  max_features=0.6, max_samples=0.2, n_estimators=5, score=0.720588 -   0.0s
[CV

[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:    5.4s


[CV]  max_features=0.8, max_samples=0.1, n_estimators=10, score=0.741722 -   0.1s
[CV] max_features=0.8, max_samples=0.1, n_estimators=10 ..............
[CV]  max_features=0.8, max_samples=0.1, n_estimators=10, score=0.748387 -   0.1s
[CV] max_features=0.8, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.8, max_samples=0.1, n_estimators=15, score=0.795031 -   0.1s
[CV] max_features=0.8, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.8, max_samples=0.1, n_estimators=15, score=0.745098 -   0.1s
[CV] max_features=0.8, max_samples=0.1, n_estimators=15 ..............
[CV]  max_features=0.8, max_samples=0.1, n_estimators=15, score=0.795031 -   0.1s
[CV] max_features=0.8, max_samples=0.2, n_estimators=5 ...............
[CV]  max_features=0.8, max_samples=0.2, n_estimators=5, score=0.748201 -   0.0s
[CV] max_features=0.8, max_samples=0.2, n_estimators=5 ...............
[CV]  max_features=0.8, max_samples=0.2, n_estimators=5, score=0.772414 -   0.0s
[C

[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed:   15.2s finished



Grid search took 15.55 seconds.
Made predictions in 0.0010 seconds.
Tuned model has a training F1 score of 0.8344.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8421.

BaggingClassifier(base_estimator=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=1001,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.2, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=1001, verbose=0, warm_start=False)


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Tuning slightly improved the test score by 1% from 0.8321 to 0.8421, but reduced the train score by 16% from .9975 to .8344.  This seems to be a sign of less overfitting to the training data.

| Tuned | F1 Score (train) | F1 Score (test) |
| :--------------------: | :--------------: | :-------------: |
|         No             |      0.9975      |     0.8321      |
|         Yes            |      0.8344      |     0.8421      |

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.